In [1]:
import datetime
import numpy as np
import pandas as pd
import json

In [2]:
with open ("product-sales.json", 'r') as json_product_data:
    product_sales = json.load (json_product_data)

In [3]:
with open ("shop-sales.json", 'r') as json_sales_data:
    shop_sales = json.load (json_sales_data)

In [4]:
shop_sales_key = {(shop_key, date_key): values
              for shop_key, shop_dict in shop_sales.items()
              for date_key, values in shop_dict.items()}

In [5]:
shop_sales_table = pd.DataFrame.from_dict(shop_sales_key, orient="index")

In [6]:
shop_sales_table['Shop'] = shop_sales_table.index.map (lambda list: list[0])
shop_sales_table['Date'] = shop_sales_table.index.map (lambda list: list[1])

In [7]:
shop_sales_table.rename(columns = {list(shop_sales_table)[0]: 'Total'}, inplace = True)

In [8]:
product_sales_key = {(shop_key, product_key, date_key): values
              for shop_key, shop_dict in product_sales.items()
              for product_key, product_dict in shop_dict.items()
              for date_key, values in product_dict.items()}

In [9]:
product_sales_table = pd.DataFrame.from_dict (product_sales_key, orient="index")

In [10]:
product_sales_table.rename(columns = {list(product_sales_table)[0]: 'Sales'}, inplace = True)

In [11]:
product_sales_table['Shop'] = product_sales_table.index.map (lambda list: list[0])
product_sales_table['Product'] = product_sales_table.index.map (lambda list: list[1])
product_sales_table['Date'] = product_sales_table.index.map (lambda list: list[2])

In [12]:
sales = pd.merge_ordered(product_sales_table, shop_sales_table, on=('Shop','Date')).fillna(0)

In [13]:
sales['datetime'] = sales['Date'].map (lambda t: datetime.datetime.strptime(t,'%Y-%m-%d'))
start_datetime = sales['datetime'].min()
end_datetime = sales['datetime'].max()

In [14]:
def recency (t):
    return (end_datetime - datetime.datetime.strptime(t,'%Y-%m-%d')).days
sales['Recency'] = sales['Date'].map(recency)

In [15]:
# Add zero counts for missing data
sales.set_index('datetime', inplace=True)
timeseries = sales.groupby(['Shop','Product'])['Sales','Total','Recency'].resample('D').sum()

# Approach: neither a clustering problem nor a supervised classification

If the product sales count in any shop, is less than a given threshold on the day, then classify as "out of stock". The threshold is calculated using a one-sided 70% confidence interval (mean - 1.04 * st.dev.), assuming a 'normal' Gaussian distribution and independence of temporal events (yesterday's sale count does not directly influence today's sale count).

If no sales count for last 90 days, then instead classify as "discontinued".

If less than 30 historical sales for any product in shop, instead classify as "unknown" i.e. limited stock.

If no sales data from shop on the day, then assume a data outage and also classify as "unknown" (limited stock).

Classify as "in stock" if neither "out of stock", nor "discontinued", nor "unknown"

In [16]:
analysis = timeseries.groupby(['Shop','Product']).agg({'Sales':['mean','std']}).fillna(0)

In [17]:
# Calculate 70% confidence interval, one-sided (z=1.04)
analysis['Threshold'] = analysis['Sales']['mean'] - 1.04 * analysis['Sales']['std']

In [18]:
recent_sales = sales.groupby(['Shop','Product']).agg({'Sales':['last','count'],
                                                      'Total':['last'],
                                                      'Recency':['min']}).fillna(0)

In [19]:
predictions = pd.merge (analysis, recent_sales, on=['Shop','Product'])

In [20]:
predictions['StockOut'] = np.where (predictions['Sales']['last'] < predictions['Threshold'],1,0)

In [21]:
predictions['Discontinued'] = np.where (predictions['Recency']['min'] > 90, 2, 0)

In [22]:
predictions['Unknown'] = np.where (predictions['Sales']['count'] < 30, 3, 0)

In [23]:
predictions['DataOutage'] = np.where (predictions['Total']['last'] == 0, 3, 0)

In [24]:
# Classify as "in stock" if neither "out of stock", nor "discontinued", nor "unknown"
predictions['Label'] = predictions[['StockOut','Discontinued','Unknown','DataOutage']].max(axis=1)
predictions['Stock status'] = np.choose(predictions['Label'],["In stock","Out of stock", "Discontinued", "Unknown"])

In [25]:
output = predictions['Stock status'].unstack('Shop').fillna("Unknown")

In [26]:
output_dict = output.to_dict()

In [27]:
with open('output.json', 'w') as f:
    json.dump(output_dict, f)

In [28]:
# The End.